In [1]:
from visual_networks import *
from sample_networks import *
import torch


# Generate some random data
x = torch.randn(50, 10)

# Initialize and run the model
model = SimpleLinearModel()
y_pred = model(x)


In [2]:
from visual import *
network = Network(model)  # Example: 3 layers with 3, 4, and 2 neurons respectively
thread = Thread(target=run_glfw_window, args=(network,))
thread.start()

-0.17774944


In [3]:
from tracking_objects import *
tracking_object()

c:\ProgramData\Miniconda3\lib\site-packages\glfw\__init__.py:916: GLFWError: (65544) b'Win32: Failed to register window class: Class already exists. '
  warnings.warn(message, GLFWError)
Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\ProgramData\Miniconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\Miniconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Michael Muratov\Documents\Programming\AI\WeightTerrain\visual.py", line 58, in run_glfw_window
    if not glfw.init():
  File "c:\ProgramData\Miniconda3\lib\site-packages\glfw\__init__.py", line 812, in init
    res = _glfw.glfwInit()
OSError: exception: access violation writing 0x0000000000000000


Exception: GLFW could not be initialized